In [ ]:
filter_operations_to: list[str] = []

In [ ]:
from code_data_science import data_table as dt, data_grid as moderne_data_grid

df = dt.read_csv("../samples/sql_crud.csv")

if len(filter_operations_to) > 0:
    df = df[df["operation"].isin(filter_operations_to)]

df = df.rename(
    columns={
        "repositoryOrigin": "Origin",
        "repositoryPath": "Path",
        "repositoryBranch": "Branch",
        "repositoryLink": "repositoryLink",
        "scmType": "scmType",
        "sourcePath": "File",
        "lineNumber": "lineNumber",
        "table": "Table",
        "column": "Column",
        "operation": "Operation",
    }
)

In [ ]:
def create_link(repositoryLink: str, scmType: str, sourcePath: str, lineNumber: str) -> str:
    """ returns a link to the source code in the repository based on scmType

    Args:
        repositoryLink (str): a url to the repository and branch
        scmType (str): the type of source control
        sourcePath (str): path from the root of project to the source code
        lineNumber (str): the line number in the source code

    Returns:
        str: link to the source code in the repository
    """
    url = repositoryLink.rstrip('/')  # Remove any trailing slash to ensure URL consistency

    if scmType == "GithubConfiguration":
        if sourcePath:
            url += f"/{sourcePath}"
            if lineNumber:
                url += f"#L{lineNumber}"

    elif scmType == "BitbucketConfiguration":
        if sourcePath:
            url += f"/{sourcePath}"
            if lineNumber:
                url += f"#{lineNumber}"
            

    elif scmType == "BitbucketCloudConfiguration":
        if sourcePath:
            url += f"/{sourcePath}"
            if lineNumber:
                url += f"#lines-{lineNumber}"

    elif scmType == "GitLabConfiguration":
        if sourcePath:
            url += f"/{sourcePath}"
            if lineNumber:
                url += f"#L{lineNumber}"

    return url

df["Link"] = ""

df["Link"] = df.apply(
    lambda row: create_link(
        row["repositoryLink"], row["scmType"], row["File"], row["lineNumber"]
    ), axis=1
)    


In [ ]:

df["Repository"] = ""
df["Organization"] = ""

# attempt to split path into two columns
try:
    df[["Organization", "Repository"]] = df["Path"].str.split("/", n=1, expand=True)
except:
    pass

df.drop(columns=["Path"], inplace=True)
df = df[
    [
        "Table",
        "Column",
        "Operation",
        "Repository",
        "File",
        "Origin",
        "Organization",
        "Branch",
        "Link",
    ]
]

df

In [ ]:
moderne_data_grid.display(df, column_types={"Link": "link"})